# [Day 22](https://adventofcode.com/2020/day/22): Crab Combat

In [1]:
import re

with open("../data/22.txt", "r") as f:
    decks = re.findall(r"Player \d:\n([\d+\n]+)\n", f.read())
    decks = (d.strip().split("\n") for d in decks)
    decks = [tuple(map(int, d)) for d in decks]

## Part 1

In [2]:
from collections import deque
from operator import mul
from itertools import starmap

def combat(decks):
    deck1, deck2 = map(deque, decks)
    while deck1 and deck2:
        if (x1 := deck1.popleft()) > (x2 := deck2.popleft()):
            deck1.extend([x1, x2])
        else:
            deck2.extend([x2, x1])
    return deck1, deck2

def score(deck):
    return sum(starmap(mul, enumerate(reversed(deck), 1)))

deck1, deck2 = combat(decks)
assert 32598 == score(deck1 or deck2)

## Part 2

In [3]:
def recurcombat(decks):
    return recurcombat_(*decks)

def recurcombat_(deck1, deck2):
    deckses = set()
    while deck1 and deck2:
        if (deck1, deck2) in deckses:
            return deck1, None  # Player 1 wins
        deckses.add((deck1, deck2))
        d1, d2 = combatround(deck1, deck2)
        if d1 and d2:
            deck1, deck2 = d1, d2
        elif d1:  # Player 1 wins round
            deck1, deck2 = accum1(deck1, deck2)
        else:     # Player 2 wins round
            deck1, deck2 = accum2(deck1, deck2)
    return deck1, deck2

def combatround(deck1, deck2):
    (x1, *d1), (x2, *d2) = deck1, deck2
    if x1 <= len(d1) and x2 <= len(d2):
        return recurcombat_(tuple(d1[:x1]), tuple(d2[:x2]))
    if x1 > x2:
        return accum1(deck1, deck2)
    return accum2(deck1, deck2)

def accum1(x, y):
    return (*x[1:], x[0], y[0]), y[1:]

def accum2(x, y):
    return x[1:], (*y[1:], y[0], x[0])

deck1, deck2 = recurcombat(decks)
assert 35836 == score(deck1 or deck2)